In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import spacy

In [2]:
code = pd.read_excel(r"/content/finalmerged.xlsx")
code

,Question,Correct_Code,Code_with_Error,Total_Marks
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0
...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0


In [3]:
# Merge columns
code['merged'] = code['Question'] + ' ' + code['Code_with_Error']
code

,Question,Correct_Code,Code_with_Error,Total_Marks,merged
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,Print the factors of a number #include <stdio....
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,Print the factors of a number #include <stdio....
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,Print the factors of a number #include <stdio....
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,Print the factors of a number #include <stdio....
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,Print the factors of a number #include <stdio....
...,...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0,Print maximum Occurring Character in a String ...
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0,Print maximum Occurring Character in a String ...


In [4]:
# Fill missing values with empty strings
code['merged'] = code['merged'].fillna('')

In [5]:
# Load spaCy model for word embeddings
nlp = spacy.load("en_core_web_sm")

In [6]:
# Tokenize and vectorize the merged data using spaCy
vectors = []
for text in code['merged']:
    doc = nlp(text)
    vectors.append(doc.vector)

In [7]:
# Split the vectors into train, test, and validation sets
X = np.array(vectors)
y = code['Total_Marks']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [13]:

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [15]:
# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/100
18/18 [==============================] - 1s 11ms/step - loss: 31.5838 - val_loss: 14.8344
Epoch 2/100
18/18 [==============================] - 0s 6ms/step - loss: 9.8179 - val_loss: 7.7788
Epoch 3/100
18/18 [==============================] - 0s 4ms/step - loss: 7.5708 - val_loss: 6.7353
Epoch 4/100
18/18 [==============================] - 0s 4ms/step - loss: 7.6647 - val_loss: 6.4834
Epoch 5/100
18/18 [==============================] - 0s 6ms/step - loss: 7.1736 - val_loss: 6.2923
Epoch 6/100
18/18 [==============================] - 0s 5ms/step - loss: 6.8013 - val_loss: 5.9983
Epoch 7/100
18/18 [==============================] - 0s 5ms/step - loss: 6.4731 - val_loss: 6.0465
Epoch 8/100
18/18 [==============================] - 0s 4ms/step - loss: 6.7679 - val_loss: 5.7536
Epoch 9/100
18/18 [==============================] - 0s 5ms/step - loss: 6.1901 - val_loss: 5.9308
Epoch 10/100
18/18 [==============================] - 0s 4ms/step - loss: 6.5983 - val_loss: 5.5206
Epoch 

In [20]:
# Make predictions on the test set
y_pred_train = model.predict(X_train).reshape(-1)
y_pred_test = model.predict(X_test).reshape(-1)

8/8 [==============================] - 0s 2ms/step


In [21]:
# Calculate metrics for train set
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
mae_train = mean_absolute_error(y_train, y_pred_train)
mape_train = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100
r2_train = r2_score(y_train, y_pred_train)

# Calculate metrics for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)
mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100
r2_test = r2_score(y_test, y_pred_test)


In [22]:
# Print the metrics
print("Train Set Metrics:")
print("RMSE:", rmse_train)
print("MAE:", mae_train)
print("MAPE:", mape_train)
print("R2 Score:", r2_train)
print("")

print("Test Set Metrics:")
print("RMSE:", rmse_test)
print("MAE:", mae_test)
print("MAPE:", mape_test)
print("R2 Score:", r2_test)

Train Set Metrics:
RMSE: 2.01380591401922
MAE: 1.6029166377736157
MAPE: inf
R2 Score: 0.16246337549099643

Test Set Metrics:
RMSE: 2.290560313187904
MAE: 1.7545300874710084
MAPE: inf
R2 Score: 0.010899731557024284
